In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the CSV file into a pandas DataFrame
correct_form = pd.read_csv("/content/drive/MyDrive/Powerlift Phone/ML Kit Data - Sheet1.csv")

# Extract values from the first column
first_column_values = correct_form.iloc[:, 0]

# Extract text after the last space for each value
processed_values = first_column_values.apply(lambda x: x.split()[-1] if isinstance(x, str) else x)

# Update the DataFrame with the processed values
correct_form.iloc[:, 0] = processed_values

# NOW WITH INCORRECT VALUES
# Read the CSV file into a pandas DataFrame
incorrect_form = pd.read_csv("/content/drive/MyDrive/Powerlift Phone/ML Kit Data - Sheet2.csv")

# Extract values from the first column
incorrect_first_column_values = incorrect_form.iloc[:, 0]

# Extract text after the last space for each value
processed_values = incorrect_first_column_values.apply(lambda x: x.split()[-1] if isinstance(x, str) else x)

# Update the DataFrame with the processed values
incorrect_form.iloc[:, 0] = processed_values

In [ ]:
print("CORRECT FORM: ")
print(correct_form)
print("------------------------------")
print("INCORRECT FORM:")
print(incorrect_form)

CORRECT FORM: 
     Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0       628.813 -114.932830  -801.21375   677.38830 -162.767520  -847.98890   
1     656.09186  -28.208960  -744.24450   697.78370  -66.806206  -781.80524   
2     676.50977  -72.184170  -807.62940   725.82550 -122.222650  -834.91425   
3     634.97675  -16.963688  -650.00854   675.44430  -61.463528  -695.31213   
4      620.2931    8.236617  -759.06573   641.72797  -34.861050  -785.35030   
...         ...         ...         ...         ...         ...         ...   
1472  410.17627  233.647430  -506.01870   429.19815  196.178470  -552.02040   
1473  406.28674  249.378980  -544.91440   426.41202  210.114650  -587.67320   
1474  407.38727  271.065550  -610.98486   429.66580  232.161290  -654.95920   
1475  406.09915  286.816930  -506.63245   429.70090  247.695050  -545.76360   
1476   408.4959  294.077670  -557.06680   434.60236  255.738830  -589.27295   

      Unnamed: 6  Unnamed: 7  Unname

In [ ]:
import numpy as np
import tensorflow as tf

X = np.asarray(pd.concat([correct_form, incorrect_form], axis=0, ignore_index=True).astype(float))

Y = np.concatenate([np.ones(len(correct_form)), np.zeros(len(incorrect_form))])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, shuffle= True)

In [ ]:
from tensorflow.keras import models, layers

def build_binary_classification_model(input_size):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=input_size))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification requires a sigmoid activation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model 1
model_1 = build_binary_classification_model(input_size=99)
model_1.fit(x_train, y_train, epochs=10, batch_size=32)

# Model 2 with a different architecture
model_2 = models.Sequential()
model_2.add(layers.Dense(128, activation='relu', input_dim=99))
#model_2.add(layers.Dense(128, activation='relu'))
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
69/69 [==============================] - 2s 3ms/step - loss: 13.2961 - accuracy: 0.7523
Epoch 2/10
69/69 [==============================] - 0s 4ms/step - loss: 2.2759 - accuracy: 0.8451
Epoch 3/10
69/69 [==============================] - 0s 3ms/step - loss: 1.8256 - accuracy: 0.8483
Epoch 4/10
69/69 [==============================] - 0s 3ms/step - loss: 1.0111 - accuracy: 0.8909
Epoch 5/10
69/69 [==============================] - 0s 3ms/step - loss: 1.1599 - accuracy: 0.8845
Epoch 6/10
69/69 [==============================] - 0s 3ms/step - loss: 0.9123 - accuracy: 0.8999
Epoch 7/10
69/69 [==============================] - 0s 3ms/step - loss: 0.7070 - accuracy: 0.9135
Epoch 8/10
69/69 [==============================] - 0s 3ms/step - loss: 0.8556 - accuracy: 0.9049
Epoch 9/10
69/69 [==============================] - 0s 3ms/step - loss: 0.5142 - accuracy: 0.9271
Epoch 10/10
69/69 [==============================] - 0s 3ms/step - loss: 0.9514 - accuracy: 0.9049
Epoch 1/10
69/69 [

In [ ]:
loss_1, accuracy_1 = model_1.evaluate(x_test, y_test)
print(f'Model 1 - Test Accuracy: {accuracy_1 * 100:.2f}%')

loss_2, accuracy_2 = model_2.evaluate(x_test, y_test)
print(f'Model 2 - Test Accuracy: {accuracy_2 * 100:.2f}%')


23/23 [==============================] - 0s 2ms/step - loss: 0.5444 - accuracy: 0.9008
Model 1 - Test Accuracy: 90.08%
23/23 [==============================] - 0s 2ms/step - loss: 0.6604 - accuracy: 0.9565
Model 2 - Test Accuracy: 95.65%


In [ ]:
converter_1 = tf.lite.TFLiteConverter.from_keras_model(model_1)
tflite_model_1 = converter_1.convert()
with open('mlkit_model_1.tflite', 'wb') as f:
    f.write(tflite_model_1)

# Convert and save model 2
converter_2 = tf.lite.TFLiteConverter.from_keras_model(model_2)
tflite_model_2 = converter_2.convert()
with open('mlkit_model_2.tflite', 'wb') as f:
    f.write(tflite_model_2)

[   0.         0.       360.4414   137.40208    0.         0.
  480.23563  152.89265    0.         0.       604.53687  470.27496
  335.33374  488.8492   653.7289   837.8279     0.         0.
  589.8475  1138.1897     0.         0.       554.59985 1127.2255
  400.2505  1115.6862   582.7978  1453.4626     0.         0.
    0.         0.       466.42474 1585.9644 ]
